In [29]:
from google.cloud import storage
import pandas as pd ## for dataset and eda
import numpy as np ## for eda
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


# %matplotlib inline
print("All good")

All good


In [30]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer # do not forget to conda install transformers
print("All packages are imported")

All packages are imported


In [31]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"GPU usage is set up: {device}")

GPU usage is set up: cuda


In [32]:
cuda.is_available()

True

In [33]:
bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

print(bucket)
print('Great, we now have access to our first bucket on google cloud storage where we put our data')

<Bucket: firstprojectdl>
Great, we now have access to our first bucket on google cloud storage where we put our data


In [34]:
from google.cloud import storage
import pandas as pd

bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "data/hw2/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')


for blob in blobs:
    print(blob.name)
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        print(file_name)
print(f"we imported the {file_name} successfully")

data/hw2/
data/hw2/nodeid2paperid.csv
nodeid2paperid.csv
data/hw2/sample.csv
sample.csv
data/hw2/testData.csv
testData.csv
data/hw2/textData.csv
textData.csv
data/hw2/trainData.csv
trainData.csv
we imported the trainData.csv successfully


In [35]:
nodeid2paperid = pd.read_csv('nodeid2paperid.csv')
nodeid2paperid.head()
nodeid2paperid.rename(columns={'node idx': 'id'}, inplace=True)
nodeid2paperid.head()

,id,paper id
0,104447,630234
1,15858,803423
2,107156,1102481
3,82077,1810480
4,42436,2131697


In [36]:
test = pd.read_csv('testData.csv')
test.head()

,id
0,137832
1,137833
2,137834
3,137836
4,137837


In [37]:
test.shape

(13718, 1)

In [38]:
text = pd.read_csv('textData.csv')
text.head()

,paper id,title,abstract
0,630234,spreadsheets on the move an evaluation of mobi...,The power of mobile devices has increased dram...
1,803423,multi view metric learning for multi view vide...,Traditional methods on video summarization are...
2,1102481,big data analytics in future internet of things,Current research on Internet of Things (IoT) m...
3,1810480,cryptographic hardening of d sequences,This paper shows how a one-way mapping using m...
4,2131697,gesture based continuous authentication for we...,We study the feasibility of touch gesture beha...


In [39]:
text.shape

(73718, 3)

In [40]:
train = pd.read_csv('trainData.csv')
train.head()

,label,id
0,4,0
1,5,1
2,8,3
3,6,6
4,4,7


In [41]:
train.shape

(60000, 2)

In [42]:
sample = pd.read_csv('sample.csv')
sample.head()

,id,label
0,137832,0
1,137833,0
2,137834,0
3,137836,0
4,137837,0


In [43]:
sample.shape

(13718, 2)

In [44]:
## Let us merge text in train and test
# train
trainData = pd.merge(train, nodeid2paperid, on='id', how='inner')

In [45]:
trainData = pd.merge(trainData, text, on ='paper id', how='inner')

In [46]:
trainData.head()

,label,id,paper id,title,abstract
0,4,0,9657784,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes..."
1,5,1,39886162,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...
2,8,3,121432379,a promise theory perspective on data networks,Networking is undergoing a transformation thro...
3,6,6,1444859417,webvrgis based city bigdata 3d visualization a...,This paper shows the WEBVRGIS platform overlyi...
4,4,7,1483430697,information theoretic authentication and secre...,"In the splitting model, information theoretic ..."


In [47]:
trainData.shape

(60000, 5)

In [48]:
# test
testData = pd.merge(test, nodeid2paperid, on='id', how='inner')

In [49]:
testData.head()

,id,paper id
0,137832,2403725649
1,137833,2404740077
2,137834,2407125866
3,137836,2408327416
4,137837,2412021890


In [50]:
testData = pd.merge(testData, text, on ='paper id', how='inner')

In [51]:
testData.head()

,id,paper id,title,abstract
0,137832,2403725649,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal..."
1,137833,2404740077,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...
2,137834,2407125866,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...
3,137836,2408327416,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...
4,137837,2412021890,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...


In [52]:
testData.shape

(13718, 4)

In [53]:
# # Removing unwanted columns and only leaving title and abstract of the scientific article and the category which will be the label
trainData = trainData[['id','title', 'abstract', 'label']]
labelListData = list(trainData['label'].unique())
trainData.head()

,id,title,abstract,label
0,0,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes...",4
1,1,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...,5
2,3,a promise theory perspective on data networks,Networking is undergoing a transformation thro...,8
3,6,webvrgis based city bigdata 3d visualization a...,This paper shows the WEBVRGIS platform overlyi...,6
4,7,information theoretic authentication and secre...,"In the splitting model, information theoretic ...",4


In [54]:
testData = testData[['id','title', 'abstract']]
testData.head()

,id,title,abstract
0,137832,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal..."
1,137833,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...
2,137834,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...
3,137836,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...
4,137837,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...


In [55]:
## merge the title and abstract columns together 
# trainData['description'] = trainData['title'].str.cat(trainData['abstract'],sep=" ")
trainData['description'] = trainData['title']

# Remove stop words in description and title ## Actually mention why you may not want to remove stop words: https://stackoverflow.com/questions/63633534/is-it-necessary-to-do-stopwords-removal-stemming-lemmatization-for-text-classif

In [56]:
trainData = trainData[['description', 'label']]

In [57]:
trainData.isna().sum()

description    0
label          0
dtype: int64

In [58]:
len(labelListData) # 20 labels and not 40

20

In [59]:
labelListData

[4, 5, 8, 6, 3, 16, 19, 14, 10, 0, 2, 18, 9, 13, 11, 1, 7, 15, 17, 12]

In [60]:
# biggest text in column
trainData['description'].apply(lambda x: len(x)).max()

255

In [61]:
trainData

,description,label
0,evasion attacks against machine learning at te...,4
1,how hard is computing parity with noisy commun...,5
2,a promise theory perspective on data networks,8
3,webvrgis based city bigdata 3d visualization a...,6
4,information theoretic authentication and secre...,4
...,...,...
59995,incentivizing users of data centers participat...,5
59996,semantic change detection with hypermaps,16
59997,computing with polynomial ordinary differentia...,9
59998,on energy efficiency in wireless networks a ga...,8


In [62]:
## testData['description'] = testData['title'].str.cat(testData['abstract'],sep=" ")
testData['description'] = testData['title']

In [63]:
testData

,id,title,abstract,description
0,137832,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal...",patchlift fast and exact computation of patch ...
1,137833,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...,the unreasonable effectiveness of address clus...
2,137834,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...,end to end goal driven web navigation
3,137836,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...,complexity measures for map reduce and compari...
4,137837,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...,a parallel implementation of the ensemble kalm...
...,...,...,...,...
13713,169336,confidence guided stereo 3d object detection w...,Accurate and reliable 3D object detection is v...,confidence guided stereo 3d object detection w...
13714,169338,sentinet detecting localized universal attacks...,SentiNet is a novel detection framework for lo...,sentinet detecting localized universal attacks...
13715,169340,learning compositional rules via neural progra...,"Many aspects of human reasoning, including lan...",learning compositional rules via neural progra...
13716,169341,certified defenses for adversarial patches,Adversarial patch attacks are among one of the...,certified defenses for adversarial patches


In [64]:
testData = testData[['description']]

In [65]:
testData

,description
0,patchlift fast and exact computation of patch ...
1,the unreasonable effectiveness of address clus...
2,end to end goal driven web navigation
3,complexity measures for map reduce and compari...
4,a parallel implementation of the ensemble kalm...
...,...
13713,confidence guided stereo 3d object detection w...
13714,sentinet detecting localized universal attacks...
13715,learning compositional rules via neural progra...
13716,certified defenses for adversarial patches


In [66]:
# biggest text in column
testData['description'].apply(lambda x: len(x)).max()

187

In [68]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 4
LEARNING_RATE = 2e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

In [69]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        description = str(self.data.description[index])
        description = " ".join(description.split())
        inputs = self.tokenizer.encode_plus(
            description,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.label[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [70]:
# Creating the dataset and dataloader for the neural network
train_size = 0.8
train_dataset= trainData.sample(frac=train_size,random_state=200)
val_dataset= trainData.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALIDATION Dataset: {}".format(val_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
validation_set = Triage(val_dataset, tokenizer, MAX_LEN)

FULL Dataset: (60000, 2)
TRAIN Dataset: (48000, 2)
VALIDATION Dataset: (12000, 2)


In [84]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **test_params)

In [78]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 20)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [79]:
model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_featu

In [80]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

In [81]:
# Function to calcuate the accuracy of the model

def calculate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [82]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%1000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 1000 steps: {loss_step}")
            print(f"Training Accuracy per 1000 steps: {accu_step}")
        print(_)
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
for epoch in tqdm(range(EPOCHS)):
    train(epoch)

  0%|          | 0/4 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length t

Training Loss per 1000 steps: 3.1071507930755615
Training Accuracy per 1000 steps: 0.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255

In [52]:
def valid(model, validation_loader):
    model.eval()
    tr_loss = 0
    nb_tr_steps = 0
    n_correct = 0 
    nb_tr_examples = 0
    with torch.no_grad():
        for _, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calculate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
            print(_)
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [53]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, validation_loader)
print("Accuracy on test data = %0.2f%%" % acc)

In [ ]:
predictions = []
with torch.no_grad():
    for _, data in tqdm(enumerate(trueTesting_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        outputs = model(ids, mask).squeeze()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        predictions.append(big_idx)

print(predictions)